**1.MCP server和client的通信**

建立连接的步骤：
1.传输层建立：通过 STDIO、Httpstreamable 或 SSE 建立双向通信通道

2.初始化握手：交换JSON-RPC报文

3.工具注册：Server 向 Client 注册可用的工具列表

4.就绪通知：双方确认初始化完成，可以开始工具调用

**2.server的三种链接方式**

STDIO,SSE（server-send-events）和httpstreamable模式

stdio：基于内存

SSE：单向通信：Server → Client 的单向流，基于HTTP长连接，使用text/event-streamContent-Type，每个消息以data:开头，以\n\n结束。

httpstreamable：双向流式通信：基于HTTP/1.1 chunked encoding，使用特殊的application/x-mcp-http-streamContent-Type，支持请求和响应的双向流式传输

httpstreamable相较于SSE的优势：

 完整的双工通信能力（SSE为单向，client向server发送信息需要额外建立连接）

 更好的错误处理和状态管理（SSE连接中断会触发重连）

 性能优势（减少连接开销：单个长连接 vs SSE可能的多连接，更低延迟：双向流减少通信回合数，更好的资源利用：更高效的网络利用率）

 如何与外部 MCP Server 通信
如果你已经有一个 MCP Client，直接提供MCPserver的url即可（本地server为http://localhost:8000）



In [3]:
import asyncio
import websockets
import json
from mcp import MCPServer, MCPClient

async def connect_to_external_server():
    # 1. 建立 WebSocket 连接
    websocket = await websockets.connect('ws://external-server:8000')
    
    # 2. 初始化你的 MCP Client
    client = MCPClient(websocket)
    
    # 3. 执行初始化握手
    await client.initialize()
    
    # 4. 现在可以调用外部 Server 的工具
    result = await client.call_tool(
        "external_tool_name", 
        {"param1": "value1"}
    )
    
    return result

ModuleNotFoundError: No module named 'mcp'

方法二：使用 MCP CLI 或 SDK
如果使用官方 MCP SDK：


In [1]:
# 通过环境变量配置外部 Server
export MCP_SERVER_URL="ws://external-server:8000"
from mcp import ClientSession, StdioServerParameters
import asyncio

async def main():
    # 配置外部 Server 连接
    server_params = StdioServerParameters(
        command="node",  # 或者直接连接 WebSocket
        args=["external-server.js"]
    )
    
    async with ClientSession(server_params) as session:
        # 获取可用工具列表
        tools = await session.list_tools()
        print("Available tools:", tools)
        
        # 调用工具
        result = await session.call_tool(
            "database_query",
            {"sql": "SELECT * FROM users LIMIT 5"}
        )
        print("Result:", result)

SyntaxError: invalid syntax (76419829.py, line 2)


方法三：使用现有的 MCP Client 库


In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent


client = MultiServerMCPClient(
    {
        "math": {
            "transport": "stdio",  # 本地子进程通信
            "command": "python",
            # 您的 math_server.py 文件的绝对路径
            "args": ["/path/to/math_server.py"],
        },
        "weather": {
            "transport": "streamable_http",  # 基于 HTTP 的远程服务器
            # 确保您在 8000 端口启动了您的天气服务器
            "url": "http://localhost:8000/mcp",
        }
    }
)

tools = await client.get_tools()
agent = create_agent(
    "anthropic:claude-sonnet-4-5",
    tools
)
math_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "what's (3 + 5) x 12?"}]}
)
weather_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "what is the weather in nyc?"}]}
)


ModuleNotFoundError: No module named 'langchain_mcp'